In [17]:
from diff_algorithm import DiffAlgorithm
from parser import DeadlyParser

file_path_after = "/Users/chloed./Documents/quivr/diff-assistant/src/cdp3/test_docs/etiquette_0_before.pdf"
file_path_before = "/Users/chloed./Documents/quivr/diff-assistant/src/cdp3/test_docs/etiquette_0_after.pdf"
complex_file = "/Users/chloed./Documents/quivr/diff-assistant/src/cdp3/test_docs/Cas3-2-3.pdf"

In [9]:
parser = DeadlyParser()
parsed_before = parser.parse(file_path_before)
parsed_after = parser.parse(file_path_after)

In [10]:
text_before = parsed_before.render()
text_after = parsed_after.render()

In [61]:
from PIL import Image
import pypdfium2 as pdfium
import torchvision.transforms as transforms

import torch
from ultralytics import YOLOv10

print("CUDA device", torch.cuda.is_available())

device = torch.device("mps")  # Default CUDA device

model = YOLOv10("./yolov10x_best.pt").to(device)

pdf = pdfium.PdfDocument(file_path_after)
page = pdf[0]  # load a page

bitmap = page.render(scale=500 / 72)

pil_image = bitmap.to_pil()

# Create a transform to convert PIL image to tensor
to_tensor = transforms.ToTensor()

# Convert PIL image to tensor (this also normalizes values to [0, 1])
tensor_image = to_tensor(pil_image)

# Add batch dimension
tensor_image = tensor_image.unsqueeze(0).to(device)

# Assuming your model is already on the CUDA device
model = model.to(device)

# Perform inference
with torch.no_grad():
    results = model.predict(source=pil_image, imgsz=1024, conf=0.35, batch=1)


annotated_image = results[0].plot()[:, :, ::-1]

im = Image.fromarray(annotated_image)

im.show()


CUDA device False

0: 1024x800 2 Pictures, 2 Section-headers, 18 Texts, 1091.6ms
Speed: 24.9ms preprocess, 1091.6ms inference, 84.6ms postprocess per image at shape (1, 3, 1024, 800)


In [47]:
print(results[0].boxes.conf)
print(results[0].boxes.cls)
results[0].boxes.xyxyn

tensor([0.8352, 0.8235, 0.8203, 0.8113, 0.7984, 0.7860, 0.6394, 0.5778, 0.5666, 0.5546, 0.5365, 0.5300, 0.4666, 0.4322, 0.4222, 0.3932, 0.3926, 0.3901], device='mps:0')
tensor([6., 9., 7., 9., 6., 9., 6., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.], device='mps:0')


In [157]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from io import BytesIO
import base64
def check_transcription(file_path, text):
    pdf = pdfium.PdfDocument(file_path)
    page = pdf[0]  # load a page
    
    bitmap = page.render(scale=500 / 72)
    
    pil_image_before = bitmap.to_pil()
    
    buffered = BytesIO()
    pil_image_before.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode()
    
    chat = ChatOpenAI(model="gpt-4o", temperature=0)
    result = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": f"Can you correct this entire text retranscription, respond only with the corrected transcription: {text}"},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{img_str}",
                            "detail": "auto",
                        },
                    },
                ]
            )
        ]
    )
    return result

In [158]:
result_before = check_transcription(file_path_before, text_before)
result_after = check_transcription(file_path_after, text_after)

In [168]:
print(result_after.content)

Coup de pâtes
TRADITION & INNOVATION

50 CREPES FINES SUCREES AU RHUM NEGRITA® (PLIEES EN QUATRE) D270 55g
50 Thin crêpes sweetened with rum Negrita® (folded in four) D270 55g

25514
Rhum NEGRITA
50 Crêpes fines sucrées au rhum cuites, surgelées -
50 Crêpes sweetened with rum, baked, frozen

Ingrédients : LAIT entier, farine de BLE, sucre de canne 16.4%, ŒUFS entiers*, beurre concentré (LAIT), eau, rhum Negrita (colorant: E150a) 3.6%, sel, poudres à lever: E500-E331-amidon de BLE.
* Œufs issus de poules élevées au sol

Ingredients : Whole MILK, WHEAT flour, cane sugar 16.4%, whole EGGS*, concentrated butter (MILK), water, Negrita rum (colouring: E150a) 3.6%, salt, raising agents: E500-E331-WHEAT starch.
* Barn eggs

Conseil d'utilisation : Décongeler le produit 1 heure entre 0° et 4°C. Après décongélation et maintien à 4°C, le produit se conserve au maximum pendant 24 heures. Suggestion: possibilité de décongeler les crêpes 30 secondes au four à micro-ondes.
How to prepare the product:

In [171]:
dmp= DiffAlgorithm()
diff_main = dmp.diff_main(result_before.content, result_after.content)
#diff_main = dmp.diff_main(text_before, text_after)
#result = dmp.to_pretty_json(diff_main, parsed_before)
diff_main

[(0, '50 CREPES FINES SUCREES AU\nCoupdegal'),
 (-1, 'g'),
 (1, 'o'),
 (0, '\nRHUM NEGRITAO (PLIEES EN QUATRE)\nTRA'),
 (-1, 'C'),
 (1, 'D'),
 (0, 'ITION '),
 (-1, '&'),
 (1, 'a'),
 (0, ' INNO'),
 (-1, 'V'),
 (1, 'Y'),
 (0, 'AT'),
 (-1, 'IG'),
 (1, ':O'),
 (0, 'N\nD270 55g\n50 Thin cr'),
 (-1, 'ê'),
 (1, 'è'),
 (0, 'pes sweetened with rum Negrita'),
 (-1, 'g'),
 (1, 'e'),
 (0, '\n(folded in four) D270 55g\n25514 R'),
 (-1, 'k'),
 (1, 'h'),
 (0, 'um'),
 (-1, 'y'),
 (0, '\n'),
 (1, 'NEGRITA '),
 (0, '50 Crêpes fines sucrées au rhum cuites, surgelées -\n'),
 (-1, 'NEGRITA\n'),
 (0, '50 Cr'),
 (-1, 'è'),
 (1, 'ê'),
 (0,
  'pes sweetened with rum, baked, frozen\nIngrédients : LAIT entier, farine de BLE, sucre de canne 16.'),
 (-1, '6'),
 (1, '4'),
 (0,
  '%, CEUFS entiers*,\nbeurre concentré (LAIT), eau, rhum Negrita (colorant: E150a) 3.'),
 (-1, '7'),
 (1, '6'),
 (0,
  '%, sel, poudres à\nlever: E500-E331-amidon de BLE.\n* CEufs issus de poules élevées au sol\nIngredients : Whole MILK, WHE

In [172]:
#split differences and send to llm 
cleaned_diff = []
for cat, content in diff_main:
    if content.strip() and content != "\n":
        cleaned_diff.append((cat, content))

In [173]:
def format_difference(main_diff):
    text_modified = ""
    sub_stack = 0
    for op, data in main_diff:
        if op == 0:  
            text_modified += data if sub_stack == 0 else f"_]] {data}"
        elif op == -1: 
            if sub_stack == 0:
                text_modified += f"[[{data}->"
                sub_stack += 1
            else:
                text_modified += f"{data}->"
        elif op == 1: 
            if sub_stack > 0:
                text_modified += f"{data}]]"
                sub_stack -= 1
            else:
                text_modified += f"[[ _ ->{data}]]"
    return text_modified

In [174]:
format_difference(cleaned_diff)

"50 CREPES FINES SUCREES AU\nCoupdegal[[g->o]]\nRHUM NEGRITAO (PLIEES EN QUATRE)\nTRA[[C->D]]ITION [[&->a]] INNO[[V->Y]]AT[[IG->:O]]N\nD270 55g\n50 Thin cr[[ê->è]]pes sweetened with rum Negrita[[g->e]]\n(folded in four) D270 55g\n25514 R[[k->h]]um[[y->NEGRITA ]]50 Crêpes fines sucrées au rhum cuites, surgelées -\n[[NEGRITA\n->_]] 50 Crè->ê]]pes sweetened with rum, baked, frozen\nIngrédients : LAIT entier, farine de BLE, sucre de canne 16.[[6->4]]%, CEUFS entiers*,\nbeurre concentré (LAIT), eau, rhum Negrita (colorant: E150a) 3.[[7->6]]%, sel, poudres à\nlever: E500-E331-amidon de BLE.\n* CEufs issus de poules élevées au sol\nIngredients : Whole MILK, WHEAT flour, cane sugar 16.[[6->4]]%, whole EGGS*, con[[c->ç]]entrated\nbutter (MILK), water, Negrita rum (colouring: E150a) 3.[[7->6]]%, salt, raising agents:\nE500-E331-WHEAT starch.\n* Barn eggs\nConseil d'utilisation : Décongeler le produit 1 heure entre 0° et 4°C. Après décongélation et\nmaintien à 4°C, le produit se conserve au maxim

In [175]:
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [176]:
section_diffs = [cleaned_diff]

In [177]:
report = []
#modified_section_names = []
for section in section_diffs:
    if len(section) == 1 and section[0][0] == 0:
        print("No differences found in this section.")
        continue
    else:
        text_modified = format_difference(section)
        #modified_section_names.append(section[0][1].split("\n")[1].split("#")[-1].strip())
        messages = [
            (
                "human",
                f"""You are tasked with analyzing and reporting differences in text for a Quality engineer. The input text contains differences marked with special tokens. Your job is to parse these differences and create a clear, concise report.

                        Here is the text containing the differences:

                        <diff_text>
                        {text_modified}
                        </diff_text>

                        RULE #1 : If there are no [[->]] tokens, it indicates no changes to report, inventing changes means death.
                        The differences are marked using the following format:
                        - [[before->after]] indicates a change from "before" to "after"
                        - If there is no "before" text, it indicates an addition
                        - If there is no "after" text, it indicates a deletion
                        - If there is no [[    ]] token, it indicates no changes to report
                        - Make sense of the difference and do not keep the '[' in the report.
                        - "_" alone means empty.

                        Follow these steps to create your report:

                        1. Carefully read through the entire text.
                        2. Identify each instance of [[ ]] tokens.
                        3. For each instance, determine the modification that was made.
                        Present your report in the following format:
                        <report>
                        In the section ..., the modification found are :
                        * the **black** cat was changed to : the **red** cat
                        * ...
                        </report>
                        Note that there might be no modifications in some sections. In that case, simply state that no differences were found.


                        Remember, your goal is to create a clear and concise report that allows the Quality engineer to quickly verify the differences. Focus on accuracy and readability in your output, give every indication possible to make it easier to find the modification.
                        The report should be written in a professional and formal tone and in French.""",
            ),
        ]
        response = llm.invoke(messages)
        report.append(response.content)

#print("The modified Sections are : ", modified_section_names)

In [178]:
print(report[0])

<report>
Dans la section "50 CREPES FINES SUCREES AU", les modifications trouvées sont :
* Coupdegal**g** a été changé en : Coupdegal**o**

Dans la section "RHUM NEGRITAO (PLIEES EN QUATRE)", les modifications trouvées sont :
* TRA**C**ITION a été changé en : TRA**D**ITION
* TRA**&** INNO**V**ATION a été changé en : TRA**a** INNO**Y**ATION
* INNO**V**ATION a été changé en : INNO**Y**ATION
* INNO**IG**N a été changé en : INNO**:O**N

Dans la section "50 Thin crêpes sweetened with rum Negrita", les modifications trouvées sont :
* cr**ê**pes a été changé en : cr**è**pes
* Negrita**g** a été changé en : Negrita**e**

Dans la section "25514 Rhum NEGRITA 50 Crêpes fines sucrées au rhum cuites, surgelées", les modifications trouvées sont :
* R**k**um a été changé en : R**h**um
* Rhum**y** a été changé en : Rhum**NEGRITA**
* NEGRITA a été changé en : (supprimé)
* Crè**ê**pes a été changé en : Crè**e**pes

Dans la section "Ingrédients", les modifications trouvées sont :
* sucre de canne 16.**6*

In [166]:
cleaned_diff

[(1, 'Coup de pâtes\nTRADITION & INNOVATION\n\n'),
 (0,
  '50 CREPES FINES SUCREES AU RHUM NEGRITA® (PLIEES EN QUATRE) D270 55g\n50 Thin crêpes sweetened with rum Negrita® (folded in four) D270 55g\n'),
 (0, '25514'),
 (0, 'Rhum NEGRITA\n50 Crêpes fines sucrées au rhum cuites, surgelées -'),
 (0, '50 Crêpes sweetened with rum, baked, frozen'),
 (0, '\nIngrédients : LAIT entier, farine de BLE, sucre de canne 16.'),
 (-1, '6'),
 (1, '4'),
 (0,
  '%, ŒUFS entiers*, beurre concentré (LAIT), eau, rhum Negrita (colorant: E150a) 3.'),
 (-1, '7'),
 (1, '6'),
 (0,
  '%, sel, poudres à lever: E500-E331-amidon de BLE.\n* Œufs issus de poules élevées au sol\n'),
 (0, 'Ingredients : Whole MILK, WHEAT flour, cane sugar 16.'),
 (-1, '6'),
 (1, '4'),
 (0,
  '%, whole EGGS*, concentrated butter (MILK), water, Negrita rum (colouring: E150a) 3.'),
 (-1, '7'),
 (1, '6'),
 (0, '%, salt, raising agents: E500-E331-WHEAT starch.\n* Barn eggs\n'),
 (0,
  "Conseil d'utilisation : Décongeler le produit 1 heure e